# fastAPI
This notebook demonstrates a simple FastAPI application implementations

In [5]:
# Basic imports
from fastapi import FastAPI, Query, Path, Body, HTTPException, status
from pydantic import BaseModel, Field
from typing import Optional, List
import uvicorn
from datetime import datetime

## Installation

Install FastAPI and Uvicorn (ASGI server):
```bash
pip install fastapi uvicorn[standard] pydantic httpx
```

## 0. Minimal FastAPI Example (Quickstart)

The simplest possible FastAPI app in just a few lines of code. This demonstrates the core concept: create an app instance, define route handlers with decorators, and return Python dictionaries (automatically converted to JSON). No complex configuration needed to get started!

In [ ]:
from fastapi import FastAPI
import uvicorn

# Create a FastAPI application instance
simple_app = FastAPI()

# Define a GET endpoint at the root path "/"
@simple_app.get("/")
def hello():
    # Return a dictionary - FastAPI automatically converts it to JSON
    return {"message": "Hello World!"}

# Define a GET endpoint with a path parameter {name}
@simple_app.get("/greet/{name}")
def greet(name: str):  # FastAPI extracts and validates the 'name' parameter
    return {"greeting": f"Hello, {name}!"}

# --- Run the server in a background thread (for notebook use) ---
import threading
import time

def run_simple():
    # Start uvicorn server on port 8001
    uvicorn.run(simple_app, host="127.0.0.1", port=8001, log_level="warning")

# Create a daemon thread so it stops when notebook closes
simple_thread = threading.Thread(target=run_simple, daemon=True)
simple_thread.start()
time.sleep(1)  # Wait for server to initialize

print("✅ Simple API running at http://127.0.0.1:8001")
print("\nTest it:")
print("  curl http://127.0.0.1:8001/")
print("  curl http://127.0.0.1:8001/greet/Alice")

✅ Simple API running at http://127.0.0.1:8001

Test it:
  curl http://127.0.0.1:8001/
  curl http://127.0.0.1:8001/greet/Alice


**That's it!** You just created a working API with:
- A root endpoint that returns JSON
- A dynamic endpoint with a path parameter

**Test it in a terminal:**
```bash
curl http://127.0.0.1:8001/
curl http://127.0.0.1:8001/greet/Bob
```

**Or save to file and run:**
```python
# Save as app.py
from fastapi import FastAPI

app = FastAPI()

@app.get("/")
def hello():
    return {"message": "Hello World!"}

@app.get("/greet/{name}")
def greet(name: str):
    return {"greeting": f"Hello, {name}!"}
```

Then run: `uvicorn app:app`

---

Now let's explore more advanced features below ⬇️

## 1. Basic FastAPI Application

Create a FastAPI app with metadata and basic endpoints. The metadata (title, description, version) automatically appears in the auto-generated API documentation. This example shows how to create simple GET endpoints that return JSON responses with dynamic data like timestamps.

In [ ]:
# Create FastAPI instance with metadata for auto-generated docs
app = FastAPI(
    title="Demo API",  # API name shown in docs
    description="FastAPI demonstration with various examples",
    version="1.0.0"  # API version
)

# Root endpoint - decorates function with @app.get() for GET requests
@app.get("/")
def read_root():
    """Root endpoint returning welcome message"""
    return {
        "message": "Welcome to FastAPI Demo",
        "timestamp": datetime.now().isoformat(),  # Current time in ISO format
        "docs": "/docs"  # Link to auto-generated documentation
    }

# Health check endpoint - useful for monitoring and load balancers
@app.get("/health")
def health_check():
    """Health check endpoint"""
    return {"status": "healthy", "timestamp": datetime.now().isoformat()}

## 2. Path Parameters

Extract values from the URL path using curly braces `{parameter_name}`. FastAPI automatically converts types (e.g., strings to integers) and validates them. You can add additional validation constraints using the `Path` function, such as minimum/maximum values, making your API more robust and self-documenting.

In [ ]:
# Path parameter example - {user_id} is extracted from URL
@app.get("/users/{user_id}")
def get_user(user_id: int):  # Type hint ensures conversion and validation
    """Get user by ID"""
    return {"user_id": user_id, "username": f"user_{user_id}"}

# Path parameter with validation constraints
@app.get("/items/{item_id}")
def get_item(
    item_id: int = Path(  # Path() adds metadata and validation
        ...,  # ... means required parameter
        title="The ID of the item",  # Documentation title
        ge=1,  # Greater than or equal to 1
        le=1000  # Less than or equal to 1000
    )
):
    """Get item with validated path parameter (between 1 and 1000)"""
    return {"item_id": item_id, "name": f"Item {item_id}"}

## 3. Query Parameters

Handle query strings that come after the `?` in URLs (e.g., `/search?q=python&limit=10`). Query parameters can be required, optional with defaults, or completely optional using `Optional[Type]`. The `Query` function allows you to add validation rules like string length, numeric ranges, and descriptions that appear in the API documentation.

In [ ]:
@app.get("/search")
def search_items(
    # Required query parameter with length validation
    q: str = Query(..., min_length=1, max_length=50, description="Search query"),
    # Optional with default value, constrained between 1 and 100
    limit: int = Query(10, ge=1, le=100, description="Number of results"),
    # Optional with default 0, must be non-negative
    skip: int = Query(0, ge=0, description="Number of items to skip")
):
    """Search with validated query parameters"""
    return {
        "query": q,
        "limit": limit,
        "skip": skip,
        # Generate sample results based on pagination
        "results": [f"Result {i} for '{q}'" for i in range(skip, skip + limit)]
    }

@app.get("/products")
def list_products(
    # All optional parameters - use Optional[type] with None default
    category: Optional[str] = None,
    min_price: Optional[float] = None,
    max_price: Optional[float] = None,
    in_stock: bool = True  # Boolean with default True
):
    """Filter products with optional parameters"""
    filters = {
        "category": category,
        "min_price": min_price,
        "max_price": max_price,
        "in_stock": in_stock
    }
    return {"filters": filters, "count": 42}

## 4. Request Body with Pydantic Models

Define data models using Pydantic for automatic request/response validation and serialization. Pydantic models provide type checking, data validation, and automatic JSON schema generation for your API docs. This eliminates manual validation code and catches errors early. The models also support nested structures, default values, and custom validation rules using `Field()`.

In [ ]:
# Define Pydantic model for data validation
class Item(BaseModel):
    # Required field with length constraints
    name: str = Field(..., min_length=1, max_length=100)
    # Optional field (None allowed) with max length
    description: Optional[str] = Field(None, max_length=500)
    # Required numeric field, must be greater than 0
    price: float = Field(..., gt=0)
    # Optional numeric field, must be >= 0 if provided
    tax: Optional[float] = Field(None, ge=0)
    # List of strings with default empty list
    tags: List[str] = []

    # Config class provides example for API documentation
    class Config:
        schema_extra = {
            "example": {
                "name": "Laptop",
                "description": "High-performance laptop",
                "price": 999.99,
                "tax": 99.99,
                "tags": ["electronics", "computers"]
            }
        }

# User model with validation
class User(BaseModel):
    username: str = Field(..., min_length=3, max_length=50)
    email: str  # Basic email string (could add email validation)
    full_name: Optional[str] = None
    age: Optional[int] = Field(None, ge=0, le=150)  # Age between 0-150

# In-memory storage dictionaries (replace with database in production)
items_db = {}
users_db = {}

# POST endpoint - creates new item with 201 status code
@app.post("/items", status_code=status.HTTP_201_CREATED)
def create_item(item: Item):  # Pydantic model automatically validates request body
    """Create a new item"""
    item_id = len(items_db) + 1
    items_db[item_id] = item.dict()  # Convert model to dictionary
    return {"item_id": item_id, **item.dict()}

# PUT endpoint - updates existing item
@app.put("/items/{item_id}")
def update_item(item_id: int, item: Item):
    """Update an existing item"""
    # Check if item exists, raise 404 if not found
    if item_id not in items_db:
        raise HTTPException(status_code=404, detail="Item not found")
    items_db[item_id] = item.dict()
    return {"item_id": item_id, **item.dict()}

# DELETE endpoint - returns 204 No Content on success
@app.delete("/items/{item_id}", status_code=status.HTTP_204_NO_CONTENT)
def delete_item(item_id: int):
    """Delete an item"""
    if item_id not in items_db:
        raise HTTPException(status_code=404, detail="Item not found")
    del items_db[item_id]
    return None  # 204 returns no content

# GET endpoint - lists all items
@app.get("/items")
def list_items():
    """List all items"""
    return {"items": items_db, "count": len(items_db)}

/tmp/ipykernel_108732/2628961089.py:2: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  class Item(BaseModel):
/home/mx/Desktop/githubYusuf3a50/wiki/python/.venv/lib/python3.11/site-packages/pydantic/_internal/_config.py:383: UserWarning: Valid config keys have changed in V2:
* 'schema_extra' has been renamed to 'json_schema_extra'
  warnings.warn(message, UserWarning)


## 5. Async Endpoints

Handle asynchronous operations using Python's `async/await` syntax. Async endpoints are particularly useful for I/O-bound operations like database queries, API calls, or file operations. While one request is waiting for I/O, the server can handle other requests, improving overall throughput. Use `async def` for the function and `await` for async operations.

In [ ]:
import asyncio

# Async endpoint - note the 'async def' instead of 'def'
@app.get("/async/slow")
async def slow_operation():
    """Simulate a slow async operation"""
    # await allows other requests to be processed during the 2-second wait
    await asyncio.sleep(2)  # Simulate database query or external API call
    return {"message": "Operation completed after 2 seconds", "timestamp": datetime.now().isoformat()}

# Async endpoint with path parameter
@app.get("/async/concurrent/{n}")
async def concurrent_tasks(n: int = Path(..., ge=1, le=10)):
    """Run multiple tasks concurrently"""
    # Define an async task
    async def task(task_id: int):
        await asyncio.sleep(1)  # Simulate I/O operation
        return f"Task {task_id} completed"
    
    # asyncio.gather runs all tasks concurrently (not sequentially)
    # This completes in ~1 second regardless of n (up to 10)
    results = await asyncio.gather(*[task(i) for i in range(n)])
    return {"tasks_run": n, "results": results}

## 6. Error Handling

Proper HTTP exception handling with meaningful error responses. Instead of Python exceptions crashing your API, use `HTTPException` to return appropriate HTTP status codes (404 Not Found, 401 Unauthorized, etc.) with descriptive error messages. This makes your API easier to debug and more user-friendly for API consumers.

In [ ]:
@app.get("/errors/not-found/{item_id}")
def get_item_with_error(item_id: int):
    """Demonstrate 404 error"""
    # Check condition and raise HTTPException with appropriate status code
    if item_id not in items_db:
        raise HTTPException(
            status_code=status.HTTP_404_NOT_FOUND,  # 404 status code
            detail=f"Item with ID {item_id} not found"  # Error message in response
        )
    return items_db[item_id]

@app.get("/errors/unauthorized")
def protected_endpoint(token: Optional[str] = None):
    """Demonstrate 401 error"""
    # Simple token validation (use proper auth in production)
    if not token or token != "secret":
        raise HTTPException(
            status_code=status.HTTP_401_UNAUTHORIZED,  # 401 status code
            detail="Invalid authentication credentials",
            headers={"WWW-Authenticate": "Bearer"}  # Standard auth header
        )
    return {"message": "Access granted", "token": token}

## 7. Running the Server & Testing with Curl

Running the FastAPI server and testing with curl commands. This section demonstrates how to start the uvicorn server in a background thread (allowing the notebook to remain interactive), and provides practical curl commands to test all the endpoints we've created. The server runs on port 8000 and provides auto-generated interactive documentation at `/docs`.

In [ ]:
import threading
import time

# Global variables to track server state
server_thread = None
server_running = False

def run_server():
    """Run uvicorn server in background"""
    global server_running
    server_running = True
    # Start uvicorn ASGI server for the FastAPI app
    uvicorn.run(app, host="127.0.0.1", port=8000, log_level="info")
    server_running = False

# Start the server only if not already running
if server_thread is None or not server_thread.is_alive():
    # Create daemon thread (stops when notebook closes)
    server_thread = threading.Thread(target=run_server, daemon=True)
    server_thread.start()
    time.sleep(2)  # Give server time to initialize
    print("✅ Server started at http://127.0.0.1:8000")
    print("📚 API docs at http://127.0.0.1:8000/docs")
    print("📖 ReDoc at http://127.0.0.1:8000/redoc")
    print("\nServer is running in the background. You can now test endpoints with curl.")
else:
    print("⚠️  Server is already running!")

INFO:     Started server process [108732]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


✅ Server started at http://127.0.0.1:8000
📚 API docs at http://127.0.0.1:8000/docs
📖 ReDoc at http://127.0.0.1:8000/redoc

Server is running in the background. You can now test endpoints with curl.


INFO:     127.0.0.1:39702 - "GET / HTTP/1.1" 200 OK
INFO:     127.0.0.1:57926 - "GET / HTTP/1.1" 200 OK
INFO:     127.0.0.1:57926 - "GET / HTTP/1.1" 200 OK


### Testing with Curl Commands

Once the server is running, open a new terminal and test the endpoints with curl:

**1. Test the root endpoint:**
```bash
curl http://127.0.0.1:8000/
```

**2. Test health check:**
```bash
curl http://127.0.0.1:8000/health
```

**3. Test path parameters:**
```bash
curl http://127.0.0.1:8000/users/42
curl http://127.0.0.1:8000/items/123
```

**4. Test query parameters:**
```bash
curl "http://127.0.0.1:8000/search?q=python&limit=5&skip=0"
curl "http://127.0.0.1:8000/products?category=electronics&min_price=100&in_stock=true"
```

**5. Test POST request (create item):**
```bash
curl -X POST http://127.0.0.1:8000/items \
  -H "Content-Type: application/json" \
  -d '{
    "name": "Laptop",
    "description": "High-performance laptop",
    "price": 999.99,
    "tax": 99.99,
    "tags": ["electronics", "computers"]
  }'
```

**6. Test PUT request (update item):**
```bash
curl -X PUT http://127.0.0.1:8000/items/1 \
  -H "Content-Type: application/json" \
  -d '{
    "name": "Updated Laptop",
    "description": "Ultra high-performance laptop",
    "price": 1299.99,
    "tax": 129.99,
    "tags": ["electronics", "computers", "premium"]
  }'
```

**7. Test DELETE request:**
```bash
curl -X DELETE http://127.0.0.1:8000/items/1
```

**8. Test database endpoints:**
```bash
# Create user
curl -X POST http://127.0.0.1:8000/db/users \
  -H "Content-Type: application/json" \
  -d '{
    "username": "johndoe",
    "email": "john@example.com",
    "full_name": "John Doe",
    "age": 30
  }'

# List all users
curl http://127.0.0.1:8000/db/users

# Get specific user
curl http://127.0.0.1:8000/db/users/1
```

**9. Test async endpoints:**
```bash
curl http://127.0.0.1:8000/async/slow
curl http://127.0.0.1:8000/async/concurrent/5
```

**10. Test error handling:**
```bash
# 404 error
curl http://127.0.0.1:8000/errors/not-found/999

# 401 error
curl http://127.0.0.1:8000/errors/unauthorized

# With valid token
curl http://127.0.0.1:8000/errors/unauthorized?token=secret
```

**Pretty print JSON responses:**
```bash
curl http://127.0.0.1:8000/ | jq
curl http://127.0.0.1:8000/db/users | jq
```

### Stopping the Server

**Method 1: From the notebook (if running in background thread):**

The server will stop automatically when you close the notebook or restart the kernel. To explicitly check status or stop:

In [19]:
# Check if server is running
if server_thread and server_thread.is_alive():
    print("✅ Server is running")
    print(f"   Thread alive: {server_thread.is_alive()}")
    print(f"   Status: {server_running}")
    print("\n🛑 To stop: Restart the kernel (Ctrl+Shift+P -> 'Restart Kernel')")
else:
    print("❌ Server is not running")

✅ Server is running
   Thread alive: True
   Status: True

🛑 To stop: Restart the kernel (Ctrl+Shift+P -> 'Restart Kernel')


## 8. Testing the API with TestClient

Testing endpoints programmatically without running a live server. FastAPI's `TestClient` uses httpx under the hood to make requests directly to your app instance. This is perfect for automated testing, CI/CD pipelines, and quick debugging. The TestClient simulates HTTP requests but executes them synchronously in-process, making tests fast and reliable.

In [ ]:
from fastapi.testclient import TestClient

# Create TestClient instance with our FastAPI app
client = TestClient(app)

# Test root endpoint
response = client.get("/")  # Makes GET request to root
print("GET /")
print(f"Status: {response.status_code}")
print(f"Response: {response.json()}\n")  # .json() parses response

# Test path parameters
response = client.get("/users/42")
print("GET /users/42")
print(f"Response: {response.json()}\n")

# Test query parameters - note the query string
response = client.get("/search?q=python&limit=5")
print("GET /search?q=python&limit=5")
print(f"Response: {response.json()}\n")

# Test POST with JSON body - json parameter auto-converts dict
response = client.post("/items", json={
    "name": "Smartphone",
    "description": "Latest model",
    "price": 699.99,
    "tax": 69.99,
    "tags": ["electronics", "mobile"]
})
print("POST /items")
print(f"Status: {response.status_code}")
print(f"Response: {response.json()}\n")

# Test list items endpoint
response = client.get("/items")
print("GET /items")
print(f"Response: {response.json()}\n")

# Test database operations (if db endpoints exist)
response = client.post("/db/users", json={
    "username": "alice",
    "email": "alice@example.com"
})
print("POST /db/users")
print(f"Status: {response.status_code}")
print(f"Response: {response.json()}\n")

response = client.get("/db/users")
print("GET /db/users")
print(f"Response: {response.json()}\n")

# Test error handling - should return 404
response = client.get("/errors/not-found/999")
print("GET /errors/not-found/999")
print(f"Status: {response.status_code}")
print(f"Response: {response.json()}")

### Option 2: Save to file and run from terminal

Save this app to `main.py` and run:
```bash
uvicorn main:app --reload --host 0.0.0.0 --port 8000
```

Then visit:
- **API**: http://localhost:8000
- **Interactive docs (Swagger)**: http://localhost:8000/docs
- **Alternative docs (ReDoc)**: http://localhost:8000/redoc

### Option 3: Run with more options
```bash
# Production mode (no auto-reload)
uvicorn main:app --host 0.0.0.0 --port 8000 --workers 4

# With custom log level
uvicorn main:app --log-level debug

# With SSL
uvicorn main:app --ssl-keyfile key.pem --ssl-certfile cert.pem
```

## Summary

This notebook demonstrated:
1. ✅ Basic FastAPI app setup
2. ✅ Path parameters with validation
3. ✅ Query parameters with defaults
4. ✅ Request bodies with Pydantic models
5. ✅ Async endpoints
6. ✅ Error handling with HTTPException
7. ✅ Database integration (SQLite)
8. ✅ Testing with TestClient
9. ✅ Running the server

**Next steps:**
- Add authentication (OAuth2, JWT)
- Implement CORS middleware
- Add background tasks
- Use SQLAlchemy ORM
- Deploy to production (Docker, Kubernetes)